In [2]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVR
lab = LabelEncoder()
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV,cross_val_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
from sklearn.ensemble import RandomForestRegressor, StackingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.impute import SimpleImputer
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, root_mean_squared_error

import warnings
warnings.filterwarnings('ignore')

data = pd.read_csv("/content/drive/MyDrive/datasets/car_price_prediction.csv")

In [3]:
data.isna().sum()

,0
ID,0
Price,0
Levy,0
Manufacturer,0
Model,0
Prod. year,0
Category,0
Leather interior,0
Fuel type,0
Engine volume,0


In [4]:
data.drop_duplicates(inplace=True)
data = data.drop(["ID", "Doors", "Color"], axis=1)

In [5]:
data.head()

,Price,Levy,Manufacturer,Model,Prod. year,Category,Leather interior,Fuel type,Engine volume,Mileage,Cylinders,Gear box type,Drive wheels,Wheel,Airbags
0,13328,1399,LEXUS,RX 450,2010,Jeep,Yes,Hybrid,3.5,186005 km,6.0,Automatic,4x4,Left wheel,12
1,16621,1018,CHEVROLET,Equinox,2011,Jeep,No,Petrol,3,192000 km,6.0,Tiptronic,4x4,Left wheel,8
2,8467,-,HONDA,FIT,2006,Hatchback,No,Petrol,1.3,200000 km,4.0,Variator,Front,Right-hand drive,2
3,3607,862,FORD,Escape,2011,Jeep,Yes,Hybrid,2.5,168966 km,4.0,Automatic,4x4,Left wheel,0
4,11726,446,HONDA,FIT,2014,Hatchback,Yes,Petrol,1.3,91901 km,4.0,Automatic,Front,Left wheel,4


In [6]:
# Levy preprocessing
data['Levy'] = data['Levy'].replace('-', np.nan).astype(float)

# Age calculation
current_date = dt.datetime.now()
data["Age"] = current_date.year - data["Prod. year"]
data = data.drop("Prod. year", axis=1)

In [7]:
# Data type conversions
data['Cylinders'] = pd.to_numeric(data['Cylinders'], errors='coerce')
data["Mileage"] = data["Mileage"].str.replace("km", "").pipe(pd.to_numeric, errors='coerce')
data["Engine volume"] = data["Engine volume"].str.replace("Turbo", "").pipe(pd.to_numeric, errors='coerce')

data.head()

,Price,Levy,Manufacturer,Model,Category,Leather interior,Fuel type,Engine volume,Mileage,Cylinders,Gear box type,Drive wheels,Wheel,Airbags,Age
0,13328,1399.0,LEXUS,RX 450,Jeep,Yes,Hybrid,3.5,186005,6.0,Automatic,4x4,Left wheel,12,15
1,16621,1018.0,CHEVROLET,Equinox,Jeep,No,Petrol,3.0,192000,6.0,Tiptronic,4x4,Left wheel,8,14
2,8467,NaN,HONDA,FIT,Hatchback,No,Petrol,1.3,200000,4.0,Variator,Front,Right-hand drive,2,19
3,3607,862.0,FORD,Escape,Jeep,Yes,Hybrid,2.5,168966,4.0,Automatic,4x4,Left wheel,0,14
4,11726,446.0,HONDA,FIT,Hatchback,Yes,Petrol,1.3,91901,4.0,Automatic,Front,Left wheel,4,11


In [8]:
# Outlier Treatment
numeric_cols = data.select_dtypes(include=['float64', 'int64']).columns
for col in numeric_cols:
    q1 = data[col].quantile(0.25)
    q3 = data[col].quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    data[col] = np.where(data[col] > upper_bound, upper_bound, data[col])
    data[col] = np.where(data[col] < lower_bound, lower_bound, data[col])

In [9]:
data.head()

,Price,Levy,Manufacturer,Model,Category,Leather interior,Fuel type,Engine volume,Mileage,Cylinders,Gear box type,Drive wheels,Wheel,Airbags,Age
0,13328.0,1399.0,LEXUS,RX 450,Jeep,Yes,Hybrid,3.5,186005.0,4.0,Automatic,4x4,Left wheel,12.0,15.0
1,16621.0,1018.0,CHEVROLET,Equinox,Jeep,No,Petrol,3.0,192000.0,4.0,Tiptronic,4x4,Left wheel,8.0,14.0
2,8467.0,NaN,HONDA,FIT,Hatchback,No,Petrol,1.3,200000.0,4.0,Variator,Front,Right-hand drive,2.0,19.0
3,3607.0,862.0,FORD,Escape,Jeep,Yes,Hybrid,2.5,168966.0,4.0,Automatic,4x4,Left wheel,0.0,14.0
4,11726.0,446.0,HONDA,FIT,Hatchback,Yes,Petrol,1.3,91901.0,4.0,Automatic,Front,Left wheel,4.0,11.0


In [10]:
data.isna().sum()

,0
Price,0
Levy,5709
Manufacturer,0
Model,0
Category,0
Leather interior,0
Fuel type,0
Engine volume,0
Mileage,0
Cylinders,0


In [11]:
data['Levy'].fillna(data['Levy'].mean(), inplace=True)
data.isna().sum()

,0
Price,0
Levy,0
Manufacturer,0
Model,0
Category,0
Leather interior,0
Fuel type,0
Engine volume,0
Mileage,0
Cylinders,0


In [12]:
data.head()

,Price,Levy,Manufacturer,Model,Category,Leather interior,Fuel type,Engine volume,Mileage,Cylinders,Gear box type,Drive wheels,Wheel,Airbags,Age
0,13328.0,1399.000000,LEXUS,RX 450,Jeep,Yes,Hybrid,3.5,186005.0,4.0,Automatic,4x4,Left wheel,12.0,15.0
1,16621.0,1018.000000,CHEVROLET,Equinox,Jeep,No,Petrol,3.0,192000.0,4.0,Tiptronic,4x4,Left wheel,8.0,14.0
2,8467.0,880.742944,HONDA,FIT,Hatchback,No,Petrol,1.3,200000.0,4.0,Variator,Front,Right-hand drive,2.0,19.0
3,3607.0,862.000000,FORD,Escape,Jeep,Yes,Hybrid,2.5,168966.0,4.0,Automatic,4x4,Left wheel,0.0,14.0
4,11726.0,446.000000,HONDA,FIT,Hatchback,Yes,Petrol,1.3,91901.0,4.0,Automatic,Front,Left wheel,4.0,11.0


In [ ]:
# Interaction Features
# data['Price_per_KM'] = data['Price'] / (data['Mileage'] + 1)
# data['Engine_per_Cylinder'] = data['Engine volume'] / data['Cylinders']
# data['Age*Mileage'] = data['Age'] * data['Mileage']

# data.head()

,Price,Levy,Manufacturer,Model,Category,Leather interior,Fuel type,Engine volume,Mileage,Cylinders,Gear box type,Drive wheels,Wheel,Airbags,Age,Price_per_KM,Engine_per_Cylinder,Age*Mileage
0,13328.0,1399.000000,LEXUS,RX 450,Jeep,Yes,Hybrid,3.5,186005.0,4.0,Automatic,4x4,Left wheel,12.0,15.0,0.071654,0.875,2790075.0
1,16621.0,1018.000000,CHEVROLET,Equinox,Jeep,No,Petrol,3.0,192000.0,4.0,Tiptronic,4x4,Left wheel,8.0,14.0,0.086567,0.750,2688000.0
2,8467.0,880.742944,HONDA,FIT,Hatchback,No,Petrol,1.3,200000.0,4.0,Variator,Front,Right-hand drive,2.0,19.0,0.042335,0.325,3800000.0
3,3607.0,862.000000,FORD,Escape,Jeep,Yes,Hybrid,2.5,168966.0,4.0,Automatic,4x4,Left wheel,0.0,14.0,0.021347,0.625,2365524.0
4,11726.0,446.000000,HONDA,FIT,Hatchback,Yes,Petrol,1.3,91901.0,4.0,Automatic,Front,Left wheel,4.0,11.0,0.127592,0.325,1010911.0


In [13]:
# Categorical Encoding
categorical_cols = ['Manufacturer', 'Category', 'Fuel type', 'Gear box type',
                    'Model', 'Leather interior', 'Wheel', 'Drive wheels']

data = pd.get_dummies(data, columns=categorical_cols, drop_first=True)

data.head()

,Price,Levy,Engine volume,Mileage,Cylinders,Airbags,Age,Manufacturer_ALFA ROMEO,Manufacturer_ASTON MARTIN,Manufacturer_AUDI,...,Model_iA isti,Model_kona,Model_macan,Model_macan S,Model_tC,Model_xD,Leather interior_Yes,Wheel_Right-hand drive,Drive wheels_Front,Drive wheels_Rear
0,13328.0,1399.000000,3.5,186005.0,4.0,12.0,15.0,False,False,False,...,False,False,False,False,False,False,True,False,False,False
1,16621.0,1018.000000,3.0,192000.0,4.0,8.0,14.0,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,8467.0,880.742944,1.3,200000.0,4.0,2.0,19.0,False,False,False,...,False,False,False,False,False,False,False,True,True,False
3,3607.0,862.000000,2.5,168966.0,4.0,0.0,14.0,False,False,False,...,False,False,False,False,False,False,True,False,False,False
4,11726.0,446.000000,1.3,91901.0,4.0,4.0,11.0,False,False,False,...,False,False,False,False,False,False,True,False,True,False


In [14]:
data.shape

(18924, 1683)

In [15]:
data.isna().sum()

,0
Price,0
Levy,0
Engine volume,0
Mileage,0
Cylinders,0
...,...
Model_xD,0
Leather interior_Yes,0
Wheel_Right-hand drive,0
Drive wheels_Front,0


In [16]:
# Prepare features and target
X = data.drop(columns=['Price'])
y = data['Price']

# Split data
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

x_train_scaled.shape, x_test_scaled.shape, y_train.shape, y_test.shape

((15139, 1682), (3785, 1682), (15139,), (3785,))

In [17]:
y_train.isna().sum()
X.isna().sum()

,0
Levy,0
Engine volume,0
Mileage,0
Cylinders,0
Airbags,0
...,...
Model_xD,0
Leather interior_Yes,0
Wheel_Right-hand drive,0
Drive wheels_Front,0


In [20]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from sklearn.preprocessing import StandardScaler
import numpy as np

# Build the neural network model
model = Sequential([
    Dense(1024, activation='relu', input_shape=(x_train.shape[1],)),  # First hidden layer
    BatchNormalization(),  # Normalize activations for stability
    Dropout(0.3),  # Prevent overfitting

    Dense(512, activation='relu'),  # Second hidden layer
    BatchNormalization(),
    Dropout(0.3),

    Dense(256, activation='relu'),  # Third hidden layer
    BatchNormalization(),
    Dropout(0.3),

    Dense(128, activation='relu'),  # Fourth hidden layer

    Dense(1)  # Output layer (for regression)
])

# Compile the model
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Train the model
model.fit(x_train_scaled, y_train, epochs=50, batch_size=32, validation_data=(x_test_scaled, y_test))

# Predict on new data
y_pred_nn = model.predict(x_test_scaled)

# Evaluate
mse_nn = np.mean((y_test - y_pred_nn.squeeze())**2)
print("Neural Network MSE:", mse_nn)

Epoch 1/50
474/474 ━━━━━━━━━━━━━━━━━━━━ 33s 56ms/step - loss: 400628704.0000 - mae: 15147.6484 - val_loss: 157687856.0000 - val_mae: 9070.8174
Epoch 2/50
474/474 ━━━━━━━━━━━━━━━━━━━━ 42s 58ms/step - loss: 127410648.0000 - mae: 8267.5000 - val_loss: 100458464.0000 - val_mae: 7031.4341
Epoch 3/50
474/474 ━━━━━━━━━━━━━━━━━━━━ 33s 42ms/step - loss: 89531416.0000 - mae: 6964.5864 - val_loss: 92102080.0000 - val_mae: 6675.7739
Epoch 4/50
474/474 ━━━━━━━━━━━━━━━━━━━━ 21s 43ms/step - loss: 84533784.0000 - mae: 6673.0327 - val_loss: 92267288.0000 - val_mae: 6574.7407
Epoch 5/50
474/474 ━━━━━━━━━━━━━━━━━━━━ 41s 43ms/step - loss: 77563888.0000 - mae: 6398.7739 - val_loss: 84835712.0000 - val_mae: 6326.7417
Epoch 6/50
474/474 ━━━━━━━━━━━━━━━━━━━━ 29s 61ms/step - loss: 71462432.0000 - mae: 6122.4204 - val_loss: 86324632.0000 - val_mae: 6427.1729
Epoch 7/50
474/474 ━━━━━━━━━━━━━━━━━━━━ 33s 45ms/step - loss: 70587784.0000 - mae: 6033.0078 - val_loss: 81650240.0000 - val_mae: 6251.3540
Epoch 8/50
474/

In [21]:
mae = mean_absolute_error(y_test, y_pred_nn)
mse = mean_squared_error(y_test, y_pred_nn)
r2 = r2_score(y_test, y_pred_nn)
rmse = root_mean_squared_error(y_test, y_pred_nn)

print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"R-squared (R²): {r2}")
print(f"Root Mean Squared Error (RMSE): {rmse}")

Mean Absolute Error (MAE): 5434.071219351805
Mean Squared Error (MSE): 66219901.705863714
R-squared (R²): 0.635721883789536
Root Mean Squared Error (RMSE): 8137.561164492941


In [22]:
import numpy as np
import pandas as pd

# Sample Car Data
car_data = pd.DataFrame([{
    "Levy": 1018,
    "Manufacturer": "CHEVROLET",
    "Model": "Equinox",
    "Age": 2025 - 2011,  # Calculate Age
    "Category": "Jeep",
    "Leather interior": "No",
    "Fuel type": "Petrol",
    "Engine volume": 3.0,
    "Mileage": 192000,  # Remove 'km'
    "Cylinders": 6.0,
    "Gear box type": "Tiptronic",
    "Drive wheels": "4x4",
    "Wheel": "Left wheel",
    "Airbags": 8
}])

# 1️⃣ Perform One-Hot Encoding (Match Training Data)
car_data = pd.get_dummies(car_data, columns=categorical_cols, drop_first=True)

# 2️⃣ Ensure Feature Alignment with Training Data
missing_cols = set(X.columns) - set(car_data.columns)
for col in missing_cols:
    car_data[col] = 0  # Add missing columns as 0

car_data = car_data[X.columns]  # Ensure column order matches training data

# 3️⃣ Standardize the Features Using the Same Scaler
car_data_scaled = scaler.transform(car_data)

# 4️⃣ Make the Prediction
predicted_price = model.predict(car_data_scaled)

print(f"Predicted Car Price: ${predicted_price[0][0]:.2f}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
Predicted Car Price: $18775.94
